<a href="https://colab.research.google.com/github/1004516/SE-ALES-Y-SISTEMAS/blob/main/EJERCICIOS%20TALLER%202/Soluciones%20Punto%201%2C8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
# ================================================================
# Taller 2 - Señales y Sistemas 2025
# Punto 1.8.1 - Dashboard Streamlit: Modulación AM y Detección Coherente
# ================================================================

!pip install streamlit
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
from numpy.fft import rfft, rfftfreq

# ------------------------------------------------------------
# Funciones auxiliares
# ------------------------------------------------------------
def butter_lowpass(cutoff, fs, order=4):
    b, a = butter(order, cutoff / (fs / 2), btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=4):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = filtfilt(b, a, data)
    return y

def spectrum(x, fs):
    X = np.abs(rfft(x))
    f = rfftfreq(len(x), 1/fs)
    return f, X / np.max(X)

# ------------------------------------------------------------
# Interfaz principal Streamlit
# ------------------------------------------------------------
st.set_page_config(page_title="Modulación AM", layout="wide")
st.title("🎚️ Modulación AM y Detección Coherente")
st.write("Interfaz interactiva para observar la modulación AM, su espectro y la detección coherente.")

# Parámetros
col1, col2, col3 = st.columns(3)
fs = col1.number_input("Frecuencia de muestreo [Hz]", 10000, 200000, 50000)
fm = col2.number_input("Frecuencia del mensaje [Hz]", 50, 5000, 200)
fc = col3.number_input("Frecuencia de la portadora [Hz]", 1000, 20000, 5000)
col4, col5 = st.columns(2)
m = col4.slider("Índice de modulación (m)", 0.1, 1.5, 0.8)
Ac = col5.number_input("Amplitud de la portadora", 0.1, 5.0, 1.0)

dur = st.slider("Duración de la simulación [s]", 0.01, 0.2, 0.05, step=0.01)
tipo = st.selectbox("Tipo de señal mensaje:", ["Senoidal", "Cuadrada", "Triangular"])

# Señal de mensaje
t = np.arange(0, dur, 1/fs)
if tipo == "Senoidal":
    msg = np.cos(2*np.pi*fm*t)
elif tipo == "Cuadrada":
    msg = np.sign(np.cos(2*np.pi*fm*t))
elif tipo == "Triangular":
    msg = 2 * np.abs(2*(t*fm - np.floor(t*fm + 0.5))) - 1
msg = msg / np.max(np.abs(msg))

# Modulación AM
y = Ac * (1 + m * msg / Ac) * np.cos(2 * np.pi * fc * t)

# Detección coherente
demod = y * np.cos(2*np.pi*fc*t)
cutoff = 3 * fm
v = butter_lowpass_filter(demod, cutoff, fs)
m_rec = (v - np.mean(v)) * (2 / m)

# Visualización
st.subheader("Visualización de señales")
tab1, tab2 = st.tabs(["Tiempo", "Frecuencia"])

with tab1:
    fig, ax = plt.subplots(3, 1, figsize=(10, 8))
    N = int(0.002*fs)
    ax[0].plot(t[:N], msg[:N]); ax[0].set_title("Mensaje m(t)")
    ax[1].plot(t[:N], y[:N]); ax[1].set_title("Señal AM")
    ax[2].plot(t[:N], m_rec[:N], label="Recuperada")
    ax[2].plot(t[:N], msg[:N], '--', label="Original", alpha=0.7)
    ax[2].set_title("Mensaje recuperado")
    for a in ax: a.grid(True)
    plt.tight_layout()
    st.pyplot(fig)

with tab2:
    f_m, M = spectrum(msg, fs)
    f_y, Y = spectrum(y, fs)
    f_r, R = spectrum(m_rec, fs)
    fig2, ax2 = plt.subplots(3, 1, figsize=(10, 8))
    ax2[0].plot(f_m, M); ax2[0].set_title("|M(f)|")
    ax2[1].plot(f_y, Y); ax2[1].set_title("|Y(f)|")
    ax2[2].plot(f_r, R); ax2[2].set_title("|Señal demodulada(f)|")
    for a in ax2: a.set_xlim(0, fc + 3*fm); a.grid(True)
    plt.tight_layout()
    st.pyplot(fig2)

st.success("✅ Simulación completada correctamente.")

2025-11-11 21:06:48.965 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-11 21:06:48.968 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-11 21:06:48.977 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-11 21:06:48.981 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-11 21:06:48.992 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-11 21:06:49.002 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-11 21:06:49.007 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-11 21:06:49.014 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()

In [26]:
# ================================================================
# Taller 2 - Señales y Sistemas 2025
# Punto 1.8.2 - Dashboard Streamlit: Cálculo de Distorsión Armónica Total (THD)
# ================================================================

import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import rfft, rfftfreq

# ------------------------------------------------------------
# Funciones auxiliares
# ------------------------------------------------------------
def spectrum(x, fs):
    X = np.abs(rfft(x))
    f = rfftfreq(len(x), 1/fs)
    return f, X / np.max(X)

def calc_thd(x, f0, fs, num_harmonics=10):
    f, X = spectrum(x, fs)
    harmonics = [np.argmin(np.abs(f - n*f0)) for n in range(1, num_harmonics + 1)]
    amps = X[harmonics]
    if len(amps) < 2:
        return 0.0
    A1 = amps[0]
    An = amps[1:]
    thd = np.sqrt(np.sum(An**2)) / A1
    return thd * 100

# ------------------------------------------------------------
# Interfaz Streamlit
# ------------------------------------------------------------
st.set_page_config(page_title="THD Calculator", layout="wide")
st.title("📊 Cálculo de Distorsión Armónica Total (THD)")
st.write("Interfaz interactiva para analizar el THD de diferentes señales periódicas.")

# Parámetros
col1, col2, col3 = st.columns(3)
fs = col1.number_input("Frecuencia de muestreo [Hz]", 10000, 200000, 50000)
f0 = col2.number_input("Frecuencia fundamental [Hz]", 50, 5000, 200)
dur = col3.slider("Duración [s]", 0.01, 0.1, 0.05, step=0.01)

tipo = st.selectbox("Forma de onda:", ["Senoidal", "Cuadrada", "Triangular", "Senoidal distorsionada"])

# Señal
t = np.arange(0, dur, 1/fs)
if tipo == "Senoidal":
    x = np.sin(2*np.pi*f0*t)
elif tipo == "Cuadrada":
    x = np.sign(np.sin(2*np.pi*f0*t))
elif tipo == "Triangular":
    x = 2 * np.abs(2*(t*f0 - np.floor(t*f0 + 0.5))) - 1
elif tipo == "Senoidal distorsionada":
    x = np.sin(2*np.pi*f0*t) + 0.3*np.sin(2*np.pi*3*f0*t) + 0.15*np.sin(2*np.pi*5*f0*t)
x = x / np.max(np.abs(x))

# Cálculo del THD
thd_val = calc_thd(x, f0, fs)

# Gráficas
st.subheader("Visualización de señal y espectro")
f, X = spectrum(x, fs)
Nplot = int(0.002 * fs)
fig, ax = plt.subplots(2, 1, figsize=(10, 7))
ax[0].plot(t[:Nplot], x[:Nplot]); ax[0].set_title("Señal en el tiempo"); ax[0].grid(True)
ax[1].plot(f, X); ax[1].set_title("Espectro de magnitud |X(f)|"); ax[1].set_xlim(0, 10*f0); ax[1].grid(True)
plt.tight_layout()
st.pyplot(fig)

st.metric(label="THD calculado", value=f"{thd_val:.2f} %")
st.success("✅ Cálculo de THD completado correctamente.")

2025-11-11 21:07:20.947 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-11 21:07:20.974 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-11 21:07:20.996 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-11 21:07:21.026 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-11 21:07:21.031 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-11 21:07:21.046 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-11 21:07:21.081 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-11 21:07:21.101 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()